In [1]:
import os

import matplotlib.pyplot as plt

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.visualization import ImageNormalize, SqrtStretch

import sunpy.coordinates  # NOQA
import sunpy.map
from sunpy.net import Fido
from sunpy.net import attrs as a

import torch

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/opt/anaconda3/envs/ML/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tutorial to download AIA images can be found there: https://docs.sunpy.org/en/stable/generated/gallery/acquiring_data/downloading_cutouts.html


In [8]:
"""
date: 2011-01-20T09:15:44.000000
basepoint_x: -226.577125	
basepoint_y: -956.964375
"""

start_time = Time('2011-01-20T09:15:44', scale='utc', format='isot')
bottom_left = SkyCoord(-126*u.arcsec, -857*u.arcsec, obstime=start_time, observer="earth", frame="helioprojective")
top_right = SkyCoord(-326*u.arcsec, -1057*u.arcsec, obstime=start_time, observer="earth", frame="helioprojective")

cutout = a.jsoc.Cutout(bottom_left, top_right=top_right, tracking=True)

#print(os.environ)
jsoc_email = 'julie.charlet@epfl.ch'

query = Fido.search(
    a.Time(start_time - 6*u.h, start_time + 6*u.h),
    a.Wavelength(304*u.angstrom),
    a.Sample(2*u.h),
    a.jsoc.Series.aia_lev1_euv_12s,
    a.jsoc.Notify(jsoc_email),
    a.jsoc.Segment.image,
    cutout,
)
print(query)

files = Fido.fetch(query)
files.sort()

Results from 1 Provider:

7 Results from the JSOCClient:
Source: http://jsoc.stanford.edu

       T_REC         TELESCOP INSTRUME WAVELNTH CAR_ROT
-------------------- -------- -------- -------- -------
2011-01-20T03:15:38Z  SDO/AIA    AIA_4      304    2105
2011-01-20T05:15:38Z  SDO/AIA    AIA_4      304    2105
2011-01-20T07:15:38Z  SDO/AIA    AIA_4      304    2105
2011-01-20T09:15:38Z  SDO/AIA    AIA_4      304    2105
2011-01-20T11:15:38Z  SDO/AIA    AIA_4      304    2106
2011-01-20T13:15:38Z  SDO/AIA    AIA_4      304    2106
2011-01-20T15:15:38Z  SDO/AIA    AIA_4      304    2106


Export request pending. [id=JSOC_20231110_952, status=2]
Waiting for 0 seconds...
Export request pending. [id=JSOC_20231110_952, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20231110_952, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20231110_952, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20231110_952, status=1]
Waiting for 5 seco

DrmsExportError:  [status=4]